In [27]:
import numpy as np
import pandas as pd 
import hvplot.pandas
from pathlib import Path

# GET DATA

In [28]:
from data import getData

In [45]:
data = getData('net', start_date='2018-01-02', end_date='2020-12-11', interval='1d', drop_columns=True)
# data = getData('AAPL')
data

,Close
Date,
2019-09-13,18.000000
2019-09-16,18.629999
2019-09-17,18.750000
2019-09-18,19.590000
2019-09-19,18.750000
...,...
2020-12-04,77.349998
2020-12-07,81.959999
2020-12-08,83.550003


# Application de la stratégie SMA

In [46]:
from Strategies.moving_averages import sma
sma(data)

In [47]:
data.tail(10)

,Close,SMA50,SMA100,Signal,Entree/Sortie
Date,,,,,
2020-11-27,74.760002,54.3126,46.2071,1.0,0.0
2020-11-30,75.080002,55.0722,46.5606,1.0,0.0
2020-12-01,72.290001,55.7388,46.8827,1.0,0.0
2020-12-02,71.900002,56.3794,47.2342,1.0,0.0
2020-12-03,74.129997,57.0624,47.6080,1.0,0.0
2020-12-04,77.349998,57.8390,48.0203,1.0,0.0
2020-12-07,81.959999,58.6774,48.4889,1.0,0.0
2020-12-08,83.550003,59.5308,48.9542,1.0,0.0
2020-12-09,77.309998,60.2636,49.3436,1.0,0.0


# PLOT

In [48]:
# Visualize exit position relative to close price
exit = data[data['Entree/Sortie'] == -1.0]['Close'].hvplot.scatter(
    color='red',
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize entry position relative to close price
entry = data[data['Entree/Sortie'] == 1.0]['Close'].hvplot.scatter(
    color='green',
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = data[['Close']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = data[['SMA50', 'SMA100']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Overlay plots
entry_exit_plot = security_close * moving_avgs * entry * exit
entry_exit_plot.opts(xaxis=None)

:Overlay
   .Curve.I          :Curve   [Date]   (Close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [Date]   (value)
   .Scatter.Close.I  :Scatter   [Date]   (Close)
   .Scatter.Close.II :Scatter   [Date]   (Close)

# BACKTEST

A partir d'ici, on va utiliser tout ce qu'il y a en dessous pour le fichier backtest

ATTENTION: backtest doit pouvoir fonctionner qu'avec les colonnes Close et Entry/Exit

In [49]:
# data = data.drop(['SMA50', 'SMA100'], axis=1)
data

,Close,SMA50,SMA100,Signal,Entree/Sortie
Date,,,,,
2019-09-13,18.000000,NaN,NaN,0.0,NaN
2019-09-16,18.629999,NaN,NaN,0.0,0.0
2019-09-17,18.750000,NaN,NaN,0.0,0.0
2019-09-18,19.590000,NaN,NaN,0.0,0.0
2019-09-19,18.750000,NaN,NaN,0.0,0.0
...,...,...,...,...,...
2020-12-04,77.349998,57.8390,48.0203,1.0,0.0
2020-12-07,81.959999,58.6774,48.4889,1.0,0.0
2020-12-08,83.550003,59.5308,48.9542,1.0,0.0


In [50]:
from backtest import application_strategie

In [51]:
application_strategie(data)
data.tail(10)

,Close,SMA50,SMA100,Signal,Entree/Sortie,Nombre de titres detenus,Nombre achat/vente,Valeur titres detenus,Espece disponible,Valeur totale du portefeuille,Rendement quotidient,Rendement cumules
Date,,,,,,,,,,,,
2020-11-27,74.760002,54.3126,46.2071,1.0,0.0,500.0,0.0,37380.001068,91000.0,128380.001068,0.018929,0.28380
2020-11-30,75.080002,55.0722,46.5606,1.0,0.0,500.0,0.0,37540.000916,91000.0,128540.000916,0.001246,0.28540
2020-12-01,72.290001,55.7388,46.8827,1.0,0.0,500.0,0.0,36145.000458,91000.0,127145.000458,-0.010853,0.27145
2020-12-02,71.900002,56.3794,47.2342,1.0,0.0,500.0,0.0,35950.000763,91000.0,126950.000763,-0.001534,0.26950
2020-12-03,74.129997,57.0624,47.6080,1.0,0.0,500.0,0.0,37064.998627,91000.0,128064.998627,0.008783,0.28065
2020-12-04,77.349998,57.8390,48.0203,1.0,0.0,500.0,0.0,38674.999237,91000.0,129674.999237,0.012572,0.29675
2020-12-07,81.959999,58.6774,48.4889,1.0,0.0,500.0,0.0,40979.999542,91000.0,131979.999542,0.017775,0.31980
2020-12-08,83.550003,59.5308,48.9542,1.0,0.0,500.0,0.0,41775.001526,91000.0,132775.001526,0.006024,0.32775
2020-12-09,77.309998,60.2636,49.3436,1.0,0.0,500.0,0.0,38654.998779,91000.0,129654.998779,-0.023498,0.29655


# PLOT POSITIONS

Ici on visualise la valeur du portefeuille tout au long des opérations d'achat et de vente

In [53]:
# Visualize exit position relative to total portfolio value
exit = data[data['Entree/Sortie'] == -1.0]['Valeur totale du portefeuille'].hvplot.scatter(
    color='red',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize entry position relative to total portfolio value
entry = data[data['Entree/Sortie'] == 1.0]['Valeur totale du portefeuille'].hvplot.scatter(
    color='green',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize total portoflio value for the investment
total_portfolio_value = data[['Valeur totale du portefeuille']].hvplot(
    line_color='lightgray',
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Overlay plots
portfolio_entry_exit_plot = total_portfolio_value * entry * exit
portfolio_entry_exit_plot.opts(xaxis=None)

:Overlay
   .Curve.I                                  :Curve   [Date]   (Valeur totale du portefeuille)
   .Scatter.Valeur_totale_du_portefeuille.I  :Scatter   [Date]   (Valeur totale du portefeuille)
   .Scatter.Valeur_totale_du_portefeuille.II :Scatter   [Date]   (Valeur totale du portefeuille)

# Portfolio evaluation 

In [54]:
from backtest import synthese_portfolio, synthese_trade

In [57]:
portfolio_evaluation_df = synthese_portfolio(data)
portfolio_evaluation_df

,Backtest
Rendements cumulés,0.33025
Rendements annuels,0.234779
Volatilité annuelle,0.099751
Ratio de Sharpe,2.353656
Ratio de Sortino,4.405167


In [56]:
portfolio_evaluation_df.reset_index(inplace=True)
portfolio_evaluation_table = portfolio_evaluation_df.hvplot.table()
portfolio_evaluation_table

:Table   [index,Backtest]

# Trade evaluation

In [58]:
from backtest import synthese_trade

In [59]:
synthese_df = synthese_trade(data, 'AAPL')
synthese_df.head()

,Symbole,Date d'entree,Date de sortie,Nombre d'actions,Prix achat d'une action,Prix de vente d'une action,Valeur totale des actions a l'achat,Valeur totale des actions a la vente,Gain/Perte


In [60]:
price_df = data[['Close', 'SMA50', 'SMA100']]
price_chart = price_df.hvplot.line()
price_chart.opts(title='Cloudflare', xaxis=None)

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [66]:
if __name__ == "__main__":
    from data import getData
    from backtest import application_strategie, synthese_portfolio, synthese_trade
    from Strategies.moving_averages import sma
    import hvplot.pandas
    
    symbol = input("Ticker Symbol:")
    start_date = input("Start Date (YYYY-MM-DD): ")
    end_date = input("End Date (YYYY-MM-DD): ")
    data = getData(symbol, start_date, end_date)

    days_for_short_sma = int(input('Time period of Short SMA (in days)'))
    days_for_long_sma = int(input('Time period of Long SMA (in days)'))

    #
    sma(data, short_window=days_for_short_sma, long_window=days_for_long_sma, make_entry_exit=True)
    application_strategie(data)
    display(data.tail(10))

    #
    portfolio_evaluation_df = synthese_portfolio(data)
    display(portfolio_evaluation_df)

    # 
    synthese_df = synthese_trade(data, symbol)
    display(synthese_df)

    ############ PLOT
    # Visualize exit position relative to close price
    exit = data[data['Entree/Sortie'] == -1.0]['Close'].hvplot.scatter(
        color='red',
        legend=False,
        ylabel='Price in $',
        width=1000,
        height=400
    )

    # Visualize entry position relative to close price
    entry = data[data['Entree/Sortie'] == 1.0]['Close'].hvplot.scatter(
        color='green',
        legend=False,
        ylabel='Price in $',
        width=1000,
        height=400
    )

    # Visualize close price for the investment
    security_close = data[['Close']].hvplot(
        line_color='lightgray',
        ylabel='Price in $',
        width=1000,
        height=400
    )

    # Visualize moving averages
    moving_avgs = data[['SMA50', 'SMA100']].hvplot(
        ylabel='Price in $',
        width=1000,
        height=400
    )

    # Overlay plots
    entry_exit_plot = security_close * moving_avgs * entry * exit
    entry_exit_plot.opts(xaxis=None)  
    hvplot.show(entry_exit_plot)  

,Close,SMA50,SMA100,Signal,Entree/Sortie,Nombre de titres detenus,Nombre achat/vente,Valeur titres detenus,Espece disponible,Valeur totale du portefeuille,Rendement quotidient,Rendement cumules
Date,,,,,,,,,,,,
2021-08-23,149.710007,142.556771,135.483963,1.0,0.0,500.0,0.0,74855.003357,71266.805649,146121.809006,0.005228,0.461218
2021-08-24,149.619995,142.943475,135.754086,1.0,0.0,500.0,0.0,74809.997559,71266.805649,146076.803207,-0.000308,0.460768
2021-08-25,148.360001,143.321754,135.982701,1.0,0.0,500.0,0.0,74180.000305,71266.805649,145446.805954,-0.004313,0.454468
2021-08-26,147.539993,143.673448,136.200026,1.0,0.0,500.0,0.0,73769.996643,71266.805649,145036.802292,-0.002819,0.450368
2021-08-27,148.600006,144.013591,136.411105,1.0,0.0,500.0,0.0,74300.003052,71266.805649,145566.808701,0.003654,0.455668
2021-08-30,153.119995,144.470695,136.642862,1.0,0.0,500.0,0.0,76559.997559,71266.805649,147826.803207,0.015525,0.478268
2021-08-31,151.830002,144.865253,136.835404,1.0,0.0,500.0,0.0,75915.000916,71266.805649,147181.806564,-0.004363,0.471818
2021-09-01,152.509995,145.239862,137.052289,1.0,0.0,500.0,0.0,76254.997253,71266.805649,147521.802902,0.002310,0.475218
2021-09-02,153.649994,145.642862,137.248776,1.0,0.0,500.0,0.0,76824.996948,71266.805649,148091.802597,0.003864,0.480918


,Backtest
Rendements cumulés,0.484168
Rendements annuels,0.089308
Volatilité annuelle,0.096061
Ratio de Sharpe,0.929703
Ratio de Sortino,1.367148


,Symbole,Date d'entree,Date de sortie,Nombre d'actions,Prix achat d'une action,Prix de vente d'une action,Valeur totale des actions a l'achat,Valeur totale des actions a la vente,Gain/Perte
0,AAPL,2017-05-25,2018-03-28,500.0,36.569141,40.026760,18284.570694,20013.380051,1728.809357
1,AAPL,2018-04-18,2018-11-30,500.0,42.758045,43.403610,21379.022598,21701.805115,322.782516
2,AAPL,2019-03-28,2020-04-07,500.0,46.064842,64.153374,23032.421112,32076.686859,9044.265747
3,AAPL,2020-06-09,2021-04-09,500.0,85.294128,132.575836,42647.064209,66287.918091,23640.853882


Launching server at http://localhost:54122
